# workflow.ipynb

# TODO
* create chart
* uniquely identify insect images
* save example dataset to GitHub
* create workbook to get data

In [5]:
import os
import glob
import shutil
import pandas as pd

In [6]:
CATALOG_PATH = '/home/aubrey/Desktop/tinian_sticky_trap_images/catalog.csv'

In [7]:
def halt_here():
    raise Exception('halt_here executed.')
    
# stop_here()

## qrcode_generator.ipynb

Generates small QRcode labels containing sequrntial serial numbers. 
Each label is placed at the center of a sticky trap to uniquely identify the trap.
The labelalso acts as a target which facilitates autofocus.
**get_image.py** can read the QRcode.
```
Usage: papermill [OPTIONS] NOTEBOOK_PATH [OUTPUT_PATH]

Parameters inferred for notebook '/home/aubrey/Desktop/sticky_trap_imaging/code/qrcode_generator.ipynb':
  FONT_SIZE: Unknown type (default 4)
                                  points
  FONT: Unknown type (default 'helvetica')
  YMIN: Unknown type (default 1)
                                  inches
  LABEL_WIDTH: Unknown type (default 0.5)
                                  inches
  LABEL_HEIGHT: Unknown type (default 0.5)
                                  inches
  XMIN: Unknown type (default 0.5)
                                  inches
  NROWS: Unknown type (default 10)
  NCOLS: Unknown type (default 10)
  PRINT: Unknown type (default False)
  MIN_SN: Unknown type (default 1)
  MAX_SN: Unknown type (default 100)
  PAGE_HEADER: Unknown type (default 'serial numbers generated by qrcode_generator.ipynb')
  PDF_FILENAME: Unknown type (default 'qrcodes.pdf')
```

## get_image.ipynb
Gets a single sticky trap image from Raspberry Pi 16MP autofocus camera.
```
Parameters inferred for notebook '/home/aubrey/Desktop/sicky_trap_imaging/code/get_image.ipynb':
  RPI_IP: Unknown type (default "192.168.1.102")
  RPI_USERNAME: Unknown type (default "pi")
  RPI_PASSWORD: Unknown type (default "canada12")
  TRAP_ID: Unknown type (default 999)
  USE_QRCODE: Unknown type (default True)
  OUTPUT_DIR: Unknown type (default './tinian_cycads')
```

## crop_square_centered_on_qrcode.ipynb
Crops a single sticky trap image using the QRcode as a center point.
```
ORIGINAL_IMAGE_PATH = '/home/aubrey/Desktop/tinian_sticky_trap_images/1/original.png'
CROPPED_IMAGE_PATH =  '/home/aubrey/Desktop/tinian_sticky_trap_images/1/cropped.png'
CROP_WIDTH = 2500
```

## extract_insect_images.ipynb
Detects insects and other particles on a sticky trap image. Each insect/particle is saved in OUTPUTPATH.
```
  IMAGEPATH: Unknown type (default '/home/aubrey/Desktop/my_sticky_trap.jpg')
  OUTPUTPATH: Unknown type (default 'my_sticky_trap')
  CANNY: Unknown type (default 45)
```

## clustimage
```
Parameters inferred for notebook '/home/aubrey/Desktop/sticky_trap_imaging/code/my_clustimage.ipynb':
  FILE_LIST_GLOB: Unknown type (default '/home/aubrey/Desktop/libcamera/tinian_cycads_cropped/40/0*a.png')
```

```
<parent>
    <trap_id>
        original.png
        cropped.png
        map.png
        1.png
        2.png
        ...
        n.png
```
* **original.png** generated by **get_image.ipynb**
* **cropped.png** generated by **crop_square_centered_on_qrcode.ipynb**
* **map.png, 1.png, 2.png** generated by **extract_insect_images.ipynb**

In [8]:
# start from scratch
command = f'rm -rf /home/aubrey/Desktop/test'
os.system(command)

0

In [9]:
# Downloading data from GitHub repo
REPO = 'https://github.com/aubreymoore/Tinian-sticky-traps-2022-02-26'
DATADIR = '/home/aubrey/Desktop/test/original'
command = 'papermill get_dataset.ipynb temp.ipynb'
if os.system(command) > 0: raise Exception('ERROR DETECTED: Open temp.ipynb for details.')

In [10]:
halt_here()

Exception: halt_here executed.

In [ ]:
print('Cropping original images.')
JUPYTER_NOTEBOOK = '/home/aubrey/Desktop/sticky_trap_imaging/code/crop_square_centered_on_qrcode.ipynb'
OUTPUT_NOTEBOOK = 'temp.ipynb'
CROP_WIDTH = 2500
for sn in range(1, 25):
    ORIGINAL_IMAGE_PATH = f'/home/aubrey/Desktop/tinian_sticky_trap_images/{sn}/original.png'
    CROPPED_IMAGE_PATH =  f'/home/aubrey/Desktop/tinian_sticky_trap_images/{sn}/cropped.png'
    print(f'   Processing{ORIGINAL_IMAGE_PATH}.')
    command = f'papermill {JUPYTER_NOTEBOOK} temp.ipynb -p ORIGINAL_IMAGE_PATH {ORIGINAL_IMAGE_PATH} -p CROPPED_IMAGE_PATH {CROPPED_IMAGE_PATH} -p CROP_WIDTH {CROP_WIDTH}'
    if os.system(command) > 0: raise Exception('ERROR DETECTED: Open temp.ipynb for details.')

In [ ]:
print('Extracting insect images.')
JUPYTER_NOTEBOOK = '/home/aubrey/Desktop/sticky_trap_imaging/code/extract_insect_images.ipynb'
OUTPUT_NOTEBOOK = 'temp.ipynb'
CANNY = 45
for sn in range(1, 25):
    IMAGEPATH = f'/home/aubrey/Desktop/tinian_sticky_trap_images/{sn}/cropped.png'
    OUTPUTPATH = f'/home/aubrey/Desktop/tinian_sticky_trap_images/{sn}'
    print(f'   Processing {IMAGEPATH}.')
    command = f'papermill {JUPYTER_NOTEBOOK} {OUTPUT_NOTEBOOK} -p IMAGEPATH {IMAGEPATH} -p OUTPUTPATH {OUTPUTPATH} -p CANNY {CANNY}'
    if os.system(command) > 0: raise Exception('ERROR DETECTED: Open temp.ipynb for details.')

In [ ]:
# Run clustimage for each size_class. Cluster will be added to catalog.
# If there is only one image in the size_class, cluster analysis is skipped and the image be in cluster -1 within the size_class

print('Running clustimage.ipynb.')
JUPYTER_NOTEBOOK = '/home/aubrey/Desktop/sticky_trap_imaging/code/clustimage.ipynb'

pd.set_option('max_colwidth', 0)
df = pd.read_csv(CATALOG_PATH)
size_classes = df['size_class'].unique().tolist()

for size_class in size_classes:
    size_class = size_class.replace(' ', '-')
    command = f'papermill {JUPYTER_NOTEBOOK} temp.ipynb -p CATALOG_PATH {CATALOG_PATH} -p SIZE_CLASS {size_class}'
    print(command)
    if os.system(command) > 0: raise Exception('ERROR DETECTED: Open temp.ipynb for details.')

In [ ]:
print('Finished')